In [ ]:
from dotenv import load_dotenv

load_dotenv()

## OpenAI functions using Langchain

### Tagging (clasification)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic
from langchain.prompts import ChatPromptTemplate
from enum import Enum
from pydantic import BaseModel, Field

In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

Fictional Scenario: A new tax reform proposal is being considered at the national level, which would impose a yearly "wealth tax" on individuals and families with a net worth exceeding a certain threshold. The tax would take a percentage of these individuals' total wealth each year, above and beyond the income tax they already pay. The proposed goal of this reform is to reduce wealth inequality and generate additional government revenue, which would be directed towards public services like education and healthcare.

Posts from Twitter:

In [ ]:
posts = [
    "User: @FreedomEagle, Followers: 130000, Likes: 8500, Shares: 2000. This new wealth tax proposal is a direct attack on the successful individuals who drive our economy. While reducing inequality is a noble goal, punishing success and hard work isn't the answer. Such measures discourage investment and slow down economic growth. We need policies that encourage wealth creation, not penalize it. As a proud conservative, I advocate for less taxation and more freedom. #NoWealthTax #EconomicFreedom",
    "User: @SocialJusticeNow, Followers: 225000, Likes: 16000, Shares: 4500. The proposed wealth tax is a step in the right direction. It's time we address the staggering wealth inequality in our nation. Asking the ultra-wealthy to contribute a small fraction of their fortune each year isn't punishment—it's their fair share. The revenue can help fund crucial public services and create more opportunities for all. As a proud progressive, I stand with this proposal. #WealthTax #FairShare",
    "User: @TheBalancedView, Followers: 150000, Likes: 12000, Shares: 3000. The wealth tax proposal brings up an important debate. On one hand, we need to address wealth inequality and fund public services. On the other hand, we must also consider the potential discouragement of investment and economic growth. Striking a balance between these concerns is key. Perhaps a compromise could be found in implementing the tax with certain protections for investments in job creation. Proud to seek the middle ground. #WealthTaxDebate #BalancedApproach",
]

In [ ]:
schema = {
    "properties": {
        "political_party": {
            "type": "string",
            "enum": ["democrats", "republicans", "independents", "others", "unknown"],
        },
        "sentiment": {"type": "string", "enum": ["positive", "neutral", "negative"]},
        "followers": {"type": "integer"},
        "user_political_orientation": {
            "type": "string",
            "enum": ["left", "center", "right", "unknown"],
        },
        "likes": {"type": "integer"},
        "shares": {"type": "integer"},
    },
    "required": [
        "political_party",
        "sentiment",
        "followers",
        "user_political_orientation",
        "likes",
        "shares",
    ],
}

In [ ]:
chain = create_tagging_chain(schema, llm)

In [ ]:
chain.prompt.messages[0]

In [ ]:
chain.run(posts[0])

In [ ]:
import pandas as pd

results = []
for post in posts:
    result = chain.run(post)
    results.append(result)

df = pd.DataFrame(results)
df

In [ ]:
class Tags(BaseModel):
    political_party: str = Field(
        ..., enum=["democrats", "republicans", "independents", "others", "unknown"]
    )
    sentiment: str = Field(..., enum=["positive", "neutral", "negative"])
    followers: int = Field(...)
    user_political_orientation: str = Field(
        ..., enum=["left", "center", "right", "unknown"]
    )
    likes: int = Field(...)
    shares: int = Field(...)

In [ ]:
pydantic_chain = create_tagging_chain_pydantic(Tags, llm)

In [ ]:
pydantic_chain.llm_kwargs

In [ ]:
tags = pydantic_chain.run(posts[0])

In [ ]:
tags

In [ ]:
tags.sentiment

### Extraction

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain, create_extraction_chain_pydantic
from langchain.prompts import ChatPromptTemplate

Fictional scenario: Extract terms from contract in the software industry to determine unfair aspects

In [ ]:
contract = """This agreement is drawn up and made effective from the first day of August, 2023, by and between XYZ Inc., a software development company ("Vendor") whose main business premises is located at 123 Silicon Valley, California, and MegaCorp ("Client"), a multinational corporation, whose principal place of business is located at 456 Wall Street, New York City, NY.

MegaCorp has expressed an interest in acquiring a custom software solution to upgrade its internal human resource management system. XYZ Inc., with its superior technological prowess, has agreed to provide the required software solution in accordance with the specifications listed in Appendix A, which forms an integral part of this agreement.

The Vendor shall complete the software solution, conduct necessary testing, and deliver the final product to the Client no later than December 31, 2023. The Client shall compensate the Vendor with an agreed amount of $1,000,000. The payment shall be made in ten equal installments, each installment becoming due at the end of each month from the signing of this agreement. 

If the Vendor fails to deliver the software solution within the stipulated time, a penalty of 5% of the total contract amount will be deducted for each week of delay. Any and all modifications requested by the Client during or after the development process shall be carried out by the Vendor free of cost.

This Agreement shall be effective for a period of 24 months from the Effective Date and may only be terminated earlier by the Client. In the event of termination, the Client shall own the rights to any and all work done by the Vendor.

Both parties shall adhere to strict confidentiality with respect to business information shared during the course of this project. Breach of this clause shall result in a penalty equivalent to the total contract amount.

All intellectual property rights in the software developed by the Vendor will belong to the Client upon full payment of the agreed fee. Any dispute arising under this Agreement shall be settled by binding arbitration in New York City, NY, following the rules set out by the MegaCorp Arbitration Association.

This agreement has been read and approved by both parties involved, indicating their acceptance of all terms and conditions."""

In [ ]:
schema = {
    "properties": {
        "parties_involved": {"type": "array", "items": {"type": "string"}},
        "effective_date": {"type": "string"},
        "duration": {"type": "integer"},
        "payment_terms": {"type": "string"},
        "service_description": {"type": "string"},
        "termination_terms": {"type": "string"},
        "confidentiality_clause": {"type": "string"},
        "intellectual_property": {"type": "string"},
        "dispute_resolution": {"type": "string"},
        "penalty_terms": {"type": "string"},
        "modifications": {"type": "string"},
    },
    "required": [
        "parties_involved",
        "effective_date",
        "payment_terms",
        "service_description",
    ],
}

In [ ]:
extraction_chain = create_extraction_chain(schema, llm)

In [ ]:
response = extraction_chain.run(contract)

In [ ]:
import pprint

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(response)

In [ ]:
extraction_chain.prompt.messages[0]

In [ ]:
extraction_chain.llm_kwargs